### ISO Specific Information

Mostly related to how version is represented and rolecode details. 

Are the rolecode URLs still available? How many are from an unexpected source? 

Do any responses reference code lists from different sources? 

How many responses (or host systems) contain a reference to a codelist that's no longer available? 

And what informational content will be affected, ie are the spatial reference blocks more likely to reference a dead code list link?
